In [1]:
import csv
import numpy as np
import pandas as pd

# Word2vec
from gensim.models import Word2Vec
from gensim.utils import tokenize
from unidecode import unidecode

In [2]:
train_data = pd.read_csv("train.csv")

In [3]:
# embedding dimension
n_dim = 20

In [4]:
sentences = [list(tokenize(s, deacc=True, lower=True)) for s in train_data['text']]
print(sentences[:5])

[['rt', 'refarcir', 'macron', 'ans', 'nom', 'prepare'], ['populaire'], ['faut', 'degager', 'cingle'], ['enseignants', 'mettre', 'prescriptions', 'president', 'republique', 'faut', 'croire', 'president', 'donner', 'periode', 'compte', 'mesures', 'appliquees', 'grever'], ['mafieuse', 'oppressive', 'macron']]


In [6]:
model = Word2Vec(size=n_dim, window=20, min_count=1, workers=7)
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=5)

15074102

In [14]:
len(model.wv.vocab)

100606

In [15]:
embeddings = np.zeros((train_data.shape[0], n_dim))

In [16]:
for idx in range(train_data.shape[0]):
    for s in train_data.loc[idx, "text"].split():
        try:
            embeddings[idx] += model.wv[unidecode(s).lower()]
        except:
            pass
            # print(idx, unidecode(s).lower())

In [17]:
df_new = pd.concat([train_data, pd.DataFrame(embeddings)], axis=1)

In [18]:
display(df_new)

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,...,10,11,12,13,14,15,16,17,18,19
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],...,-10.329475,2.416650,1.502693,2.850719,-2.807256,-1.232102,9.731252,2.610577,-2.143847,-6.653139
1,populaire,0,0,86,1016,284,[],[],0,[],...,-1.944951,3.484553,4.571080,-5.363914,-0.898058,-1.820957,0.197889,2.877736,-0.932848,1.828925
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],...,-0.435178,3.945967,0.362299,2.741020,-1.542149,-2.461654,10.717872,6.791409,-7.489476,-4.954580
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],...,-19.358449,22.202632,20.250602,3.460201,-3.469001,5.398190,27.191755,9.392954,-10.310038,-18.302785
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],...,-1.574066,0.049060,-1.081434,0.932343,-0.047059,-3.023657,2.956547,0.888699,0.010338,-1.387658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353964,gonflette tour raciste frustré,0,0,34,1509,55,[],['https://t.co/pma33zhslx'],0,[],...,0.219590,4.482287,7.534699,-0.570174,-7.077510,-8.073620,6.754254,3.285620,3.923032,-0.467246
353965,france caste crapuleuse encadrée gangsters irr...,0,0,89,11166,127,[],[],0,[],...,-0.452860,0.799088,3.225183,-5.057557,-0.339059,-1.494146,4.552934,5.059692,-2.657915,-3.700559
353966,eric zemmour français berbère,3,0,1888,712,3086,[],[],0,[],...,-3.419674,6.901509,-0.914636,3.637993,3.022601,-13.875985,2.181539,16.878120,6.158353,-1.676198
353967,gauchistes dépression pq,0,0,139,486,320,[],[],0,[],...,2.300402,-1.037353,7.092246,0.432810,-6.261333,0.504664,0.808979,3.833079,-1.410001,-2.460617


In [25]:
df_new.to_csv("train_with_embeddings.csv")